### Docs

https://github.com/aws/amazon-sagemaker-examples/blob/main/serverless-inference/serverless-model-registry.ipynb

# 1.0 Imports

In [35]:
import pandas as pd
import numpy as np
import random
import os
import time
from sklearn.model_selection import train_test_split
import boto3
import sagemaker
from sagemaker.estimator import Estimator
from datetime import datetime
from time import gmtime, strftime

# 2.0 Dataset Generation

In [169]:
weather_categories = ['SUNNY', 'CLOUDY', 'RAINY', 'FOG', 'SNOW']

coordinates = [ {'latitude': -23.5505, 'longitude': -46.6333},
                {'latitude': 40.7128, 'longitude': -74.0060},
                {'latitude': 51.5074, 'longitude': -0.1278},
                {'latitude': 35.6895, 'longitude': 139.6917},
                {'latitude': -33.8688, 'longitude': 151.2093}]

def generate_example():
    example = {}
    example['PREDICT_WEATHER_CATEGORY'] = random.choice(weather_categories)
    coordinates_choice = random.choice(coordinates)
    example['LATITUDE'] = coordinates_choice['latitude']
    example['LONGITUDE'] = coordinates_choice['longitude']
    example['TIMESTAMP'] = random.randint(int(time.time()) - 86400*365, int(time.time()))  # Timestamp Unix aleatório dentro de 1 ano
    
    return example

# Gera 100 exemplos de dados
dataset = [generate_example() for _ in range(10000)]

index_to_class = {0: 'CLOUDY', 1: 'FOG', 2: 'RAINY', 3: 'SNOW', 4: 'SUNNY'}
CLASS_INDEX = {'CLOUDY': 0, 'FOG': 1, 'RAINY': 2, 'SNOW': 3, 'SUNNY': 4}

dataset_df = pd.DataFrame(dataset)
dataset_df['PREDICT_WEATHER_CATEGORY'] = dataset_df['PREDICT_WEATHER_CATEGORY'].map(CLASS_INDEX)
dataset_df

,PREDICT_WEATHER_CATEGORY,LATITUDE,LONGITUDE,TIMESTAMP
0,3,51.5074,-0.1278,1690772130
1,3,40.7128,-74.0060,1698249241
2,1,40.7128,-74.0060,1714222986
3,0,51.5074,-0.1278,1698835558
4,1,40.7128,-74.0060,1700460952
...,...,...,...,...
9995,0,40.7128,-74.0060,1711432537
9996,3,35.6895,139.6917,1708491055
9997,1,35.6895,139.6917,1692341193
9998,3,51.5074,-0.1278,1718628019


# 3. 0 Feature Engineering

## 3.1 Time

In [170]:
def transform_dataset(dataset_df):
    """
    Transforms the original DataFrame by converting UNIX timestamps to datetime and extracting additional time-based features.

    Parameters:
    dataset_df (pd.DataFrame): The original DataFrame containing a 'TIMESTAMP' column with UNIX timestamps.

    Returns:
    pd.DataFrame: The transformed DataFrame with additional columns for date and time features.
    """

    def convert_unix_timestamp(timestamp):
        """
        Converts a UNIX timestamp to a datetime object.

        Parameters:
        timestamp (int): The UNIX timestamp to convert.

        Returns:
        datetime: The corresponding datetime object.
        """
        return datetime.fromtimestamp(timestamp)

    # Convert the UNIX timestamps to datetime objects
    dataset_df['DATE'] = dataset_df['TIMESTAMP'].apply(convert_unix_timestamp)

    # Extract additional time-based features from the 'DATE' column
    dataset_df['DAY'] = dataset_df['DATE'].dt.day
    dataset_df['MONTH'] = dataset_df['DATE'].dt.month
    dataset_df['YEAR'] = dataset_df['DATE'].dt.year
    dataset_df['WEEK_OF_YEAR'] = dataset_df['DATE'].dt.isocalendar().week
    dataset_df['HOUR'] = dataset_df['DATE'].dt.hour
    
    # Excluding unecessary columns
    dataset_df = dataset_df.drop(columns = ['DATE', 'TIMESTAMP'])

    return dataset_df

dataset_fe = transform_dataset(dataset_df)
dataset_fe

,PREDICT_WEATHER_CATEGORY,LATITUDE,LONGITUDE,DAY,MONTH,YEAR,WEEK_OF_YEAR,HOUR
0,3,51.5074,-0.1278,31,7,2023,31,2
1,3,40.7128,-74.0060,25,10,2023,43,15
2,1,40.7128,-74.0060,27,4,2024,17,13
3,0,51.5074,-0.1278,1,11,2023,44,10
4,1,40.7128,-74.0060,20,11,2023,47,6
...,...,...,...,...,...,...,...,...
9995,0,40.7128,-74.0060,26,3,2024,13,5
9996,3,35.6895,139.6917,21,2,2024,8,4
9997,1,35.6895,139.6917,18,8,2023,33,6
9998,3,51.5074,-0.1278,17,6,2024,25,12


# 4.0 Feature Split

In [171]:
# Assuming you already have a dataset where 'target' is your target variable
# and 'features' are your input features.

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset_fe.drop(columns=['PREDICT_WEATHER_CATEGORY']), 
                                                    dataset_fe['PREDICT_WEATHER_CATEGORY'], 
                                                    test_size=0.2, 
                                                    random_state=42)

# Concatenar X_train e y_train em um único DataFrame
train_data = pd.concat([y_train, X_train], axis=1)

# Concatenar X_test e y_test em um único DataFrame
test_data = pd.concat([y_test, X_test], axis=1)

# Salvar os conjuntos de dados de treinamento e teste juntos em um único arquivo CSV
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [172]:
# Bot client
s3 = boto3.client('s3')

# Diretório onde os arquivos CSV estão localizados
directory = f"{os.getcwd()}/"

# Conecte-se ao cliente S3
s3 = boto3.client('s3')

# Upload do arquivo train.csv
train_file = os.path.join(directory, 'train.csv')
s3.upload_file(train_file, 'weatherwizard-ml-model', 'data/train.csv')

# Upload do arquivo test.csv
test_file = os.path.join(directory, 'test.csv')
s3.upload_file(test_file, 'weatherwizard-ml-model', 'data/test.csv')

# 5.0 Model Setup

In [173]:
# Setup clients
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [174]:
boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
base_job_prefix = "xgboost-weatherwizard-ml-model"
role = sagemaker.get_execution_role()
print(role)

#default_bucket = f'weatherwizard-ml-model-{sagemaker_session.default_bucket()}'
default_bucket = 'weatherwizard-ml-model'
s3_prefix = base_job_prefix

training_instance_type = "ml.m5.xlarge"

us-east-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::279288473542:role/service-role/AmazonSageMaker-ExecutionRole-20240313T183292


## 5.1 Model Training

In [175]:
from sagemaker.inputs import TrainingInput

training_path = f"s3://{default_bucket}/data/train.csv"
train_input = TrainingInput(training_path, content_type="text/csv")

In [187]:
model_path = f"s3://{default_bucket}/{s3_prefix}/xgb_model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(framework="xgboost",
                                          region=region,
                                          version="1.0-1",
                                          py_version="py3",
                                          instance_type=training_instance_type,)
# Configure Training Estimator
xgb_train = Estimator(image_uri=image_uri,
                      instance_type=training_instance_type,
                      instance_count=1,
                      output_path=model_path,
                      sagemaker_session=sagemaker_session,
                      role=role)
# Set Hyperparameters
xgb_train.set_hyperparameters(objective="multi:softmax",  # Alterado para classificação multiclasse
                              num_class=5,  # Defina o número de classes esperadas
                              num_round=50,
                              max_depth=5,
                              eta=0.2,
                              gamma=4,
                              min_child_weight=6,
                              subsample=0.7,
                              silent=0,
                              eval_metric="f1")

In [188]:
# Fit model
xgb_train.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-06-22-18-16-32-343


2024-06-22 18:16:32 Starting - Starting the training job...
2024-06-22 18:16:56 Starting - Preparing the instances for training...
2024-06-22 18:17:21 Downloading - Downloading input data...
2024-06-22 18:17:46 Downloading - Downloading the training image...
2024-06-22 18:18:27 Training - Training image download completed. Training in progress...[2024-06-22 18:18:38.061 ip-10-0-217-71.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter eval_metric value f1 to Json.
Returning the value itself
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Dete

In [189]:
# Retrieve model data from training job
model_artifacts = xgb_train.model_data
model_artifacts

's3://weatherwizard-ml-model/xgboost-weatherwizard-ml-model/xgb_model/sagemaker-xgboost-2024-06-22-18-16-32-343/output/model.tar.gz'

# 6.0 Model Registry

In [190]:
# Create a Model Package Group: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-model-group.html
model_package_group_name = "xgboost-weather-wizard" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_package_group_input_dict = {"ModelPackageGroupName": model_package_group_name,
                                  "ModelPackageGroupDescription": "Model package group for xgboost regression model"}

create_model_pacakge_group_response = client.create_model_package_group(**model_package_group_input_dict)
print("ModelPackageGroup Arn : {}".format(create_model_pacakge_group_response["ModelPackageGroupArn"]))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:279288473542:model-package-group/xgboost-weather-wizard2024-06-22-18-19-15


In [191]:
model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
modelpackage_inference_specification = {"InferenceSpecification": {"Containers": [{"Image": image_uri}],
                                        "SupportedContentTypes": ["text/csv"],
                                        "SupportedResponseMIMETypes": ["text/csv"]}}

# Specify the model source
model_url = model_artifacts

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"] = model_url

create_model_package_input_dict = {"ModelPackageGroupName": model_package_group_arn,
                                   "ModelPackageDescription": "Model for regression Weather Wizard dataset",
                                   "ModelApprovalStatus": "PendingManualApproval"}

create_model_package_input_dict.update(modelpackage_inference_specification)

# Create cross-account model package
create_mode_package_response = client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print("ModelPackage Version ARN : {}".format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-06-22-18-19-15/1


In [192]:
client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'xgboost-weather-wizard2024-06-22-18-19-15',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-06-22-18-19-15/1',
   'ModelPackageDescription': 'Model for regression Weather Wizard dataset',
   'CreationTime': datetime.datetime(2024, 6, 22, 18, 19, 16, 904000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': 'e0d66dab-1f65-4305-af11-4cb8d1d9bd5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e0d66dab-1f65-4305-af11-4cb8d1d9bd5d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '425',
   'date': 'Sat, 22 Jun 2024 18:19:17 GMT'},
  'RetryAttempts': 0}}

In [193]:
model_package_arn = client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"][0]["ModelPackageArn"]
model_package_arn

'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-06-22-18-19-15/1'

In [194]:
client.describe_model_package(ModelPackageName=model_package_arn)

{'ModelPackageGroupName': 'xgboost-weather-wizard2024-06-22-18-19-15',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-06-22-18-19-15/1',
 'ModelPackageDescription': 'Model for regression Weather Wizard dataset',
 'CreationTime': datetime.datetime(2024, 6, 22, 18, 19, 16, 904000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
    'ModelDataUrl': 's3://weatherwizard-ml-model/xgboost-weatherwizard-ml-model/xgb_model/sagemaker-xgboost-2024-06-22-18-16-32-343/output/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanStatuses': []},
 'CertifyForMarketplace': 

In [195]:
# Approve the model package
model_package_update_input_dict = {"ModelPackageArn": model_package_arn,
                                   "ModelApprovalStatus": "Approved"}

model_package_update_response = client.update_model_package(**model_package_update_input_dict)
print(model_package_update_response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:279288473542:model-package/xgboost-weather-wizard2024-06-22-18-19-15/1', 'ResponseMetadata': {'RequestId': '5d54cd41-494d-46ab-a8e7-6d257fe2b390', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5d54cd41-494d-46ab-a8e7-6d257fe2b390', 'content-type': 'application/x-amz-json-1.1', 'content-length': '120', 'date': 'Sat, 22 Jun 2024 18:19:19 GMT'}, 'RetryAttempts': 0}}


# 7.0 Deployment

## 7.1 Model Registration

In [196]:
model_name = "xgboost-weather-wizard-serverless-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name : {}".format(model_name))
container_list = [{"ModelPackageName": model_package_arn}]
create_model_response = client.create_model(ModelName=model_name, ExecutionRoleArn=role, Containers=container_list)
print("Model arn : {}".format(create_model_response["ModelArn"]))

Model name : xgboost-weather-wizard-serverless-model2024-06-22-18-19-23
Model arn : arn:aws:sagemaker:us-east-1:279288473542:model/xgboost-weather-wizard-serverless-model2024-06-22-18-19-23


## 7.2 Endpoint Configuration

This is where you can adjust the Serverless Configuration for your endpoint. The current max concurrent invocations for a single endpoint, known as MaxConcurrency, can be any value from 1 to 200, and MemorySize can be any of the following: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB.

In [197]:
try:
    endpoint_config_name = "xgboost-serverless-weather-wizard-endpoint-configuration"# + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    print(endpoint_config_name)
    create_endpoint_config_response = client.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{"ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                             "ModelName": model_name,
                             "VariantName": "AllTraffic"}])
    print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
    
except:
    print("Endpoint configuration already created")

xgboost-serverless-weather-wizard-endpoint-configuration
Endpoint configuration already created


In [198]:
endpoint_config_name = "xgboost-serverless-weather-wizard-endpoint-configuration"

def create_or_update_endpoint_config(client, endpoint_config_name, model_name):
    try:
        # Verifique se o endpoint configuration já existe
        existing_configs = client.list_endpoint_configs(NameContains=endpoint_config_name)
        for config in existing_configs['EndpointConfigs']:
            if config['EndpointConfigName'] == endpoint_config_name:
                print(f"Endpoint configuration '{endpoint_config_name}' already exists. Updating it.")

                # Delete o endpoint configuration existente
                client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                print(f"Deleted existing endpoint configuration '{endpoint_config_name}'.")
                break

        # Crie o novo endpoint configuration
        create_endpoint_config_response = client.create_endpoint_config(
            EndpointConfigName=endpoint_config_name,
            ProductionVariants=[{"ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                                 "ModelName": model_name,
                                 "VariantName": "AllTraffic",}])
        
        print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
    except client.exceptions.ResourceNotFound:
        # Se o endpoint configuration não existe, crie um novo
        print(f"Endpoint configuration '{endpoint_config_name}' does not exist. Creating it.")
        create_endpoint_config_response = client.create_endpoint_config(EndpointConfigName=endpoint_config_name,
                                                                        ProductionVariants=[{"ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
                                                                                             "ModelName": model_name,
                                                                                             "VariantName": "AllTraffic"}])
        print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])
        
    except Exception as e:
        print(f"An error occurred: {e}")

create_or_update_endpoint_config(client, endpoint_config_name, model_name)

Endpoint configuration 'xgboost-serverless-weather-wizard-endpoint-configuration' already exists. Updating it.
Deleted existing endpoint configuration 'xgboost-serverless-weather-wizard-endpoint-configuration'.
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:279288473542:endpoint-config/xgboost-serverless-weather-wizard-endpoint-configuration


## 7.3 Endpoint Creation

In [199]:
endpoint_name = "xgboost-weather-wizard-serverless-endpoint"

def create_or_update_endpoint(client, endpoint_name, endpoint_config_name):
    try:
        print(f"Checking if the endpoint '{endpoint_name}' already exists.")
        existing_endpoints = client.list_endpoints(NameContains=endpoint_name)
        
        for endpoint in existing_endpoints['Endpoints']:
            if endpoint['EndpointName'] == endpoint_name:
                print(f"Endpoint '{endpoint_name}' already exists. Preparing to update it.")

                print(f"Deleting the existing endpoint '{endpoint_name}'.")
                client.delete_endpoint(EndpointName=endpoint_name)

                print(f"Waiting for the endpoint '{endpoint_name}' to be completely deleted.")
                waiter = client.get_waiter('endpoint_deleted')
                waiter.wait(EndpointName=endpoint_name)
                print(f"Endpoint '{endpoint_name}' has been successfully deleted.")
                break

        print(f"Creating a new endpoint with name '{endpoint_name}'.")
        create_endpoint_response = client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
        print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    
    except client.exceptions.ResourceNotFound:
        print(f"Endpoint '{endpoint_name}' does not exist. Creating it.")
        create_endpoint_response = client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name
        )
        print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    
    except Exception as e:
        print(f"An error occurred: {e}")

create_or_update_endpoint(client, endpoint_name, endpoint_config_name)


Checking if the endpoint 'xgboost-weather-wizard-serverless-endpoint' already exists.
Endpoint 'xgboost-weather-wizard-serverless-endpoint' already exists. Preparing to update it.
Deleting the existing endpoint 'xgboost-weather-wizard-serverless-endpoint'.
Waiting for the endpoint 'xgboost-weather-wizard-serverless-endpoint' to be completely deleted.
Endpoint 'xgboost-weather-wizard-serverless-endpoint' has been successfully deleted.
Creating a new endpoint with name 'xgboost-weather-wizard-serverless-endpoint'.
Endpoint Arn: arn:aws:sagemaker:us-east-1:279288473542:endpoint/xgboost-weather-wizard-serverless-endpoint


In [200]:
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'xgboost-weather-wizard-serverless-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:279288473542:endpoint/xgboost-weather-wizard-serverless-endpoint',
 'EndpointConfigName': 'xgboost-serverless-weather-wizard-endpoint-configuration',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost@sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
     'ResolutionTime': datetime.datetime(2024, 6, 22, 18, 20, 8, 554000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 1024, 'MaxConcurrency': 10}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 6, 22, 18, 20, 7, 8860

# 8.0 Invocation

In [201]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b"40.7128,	-74.006, 9, 1, 2024, 2, 11",
    ContentType="text/csv",
)

#print(response["Body"].read())
index_to_class[np.round(float(response["Body"].read().decode('utf-8')), 0)]

'FOG'

In [204]:
dataset_fe = dataset_fe.drop(columns = ['PREDICT_WEATHER_CATEGORY'])

In [205]:
for i in range(100):
    numpy_df_event = dataset_fe.iloc[i].to_numpy()
    data_to_pred = [str(element) for element in numpy_df_event]
    formatted_str = ','.join(data_to_pred)
    formatted_str = formatted_str.replace(' ', '')
    bytes_arr = bytes(formatted_str, 'utf-8')
    response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                Body=bytes_arr,
                                ContentType="text/csv")
    #print(response["Body"].read())
    print(index_to_class[np.round(float(response["Body"].read().decode('utf-8')), 0)])

FOG
FOG
SNOW
FOG
CLOUDY
CLOUDY
SNOW
SUNNY
SNOW
SUNNY
SNOW
CLOUDY
SUNNY
CLOUDY
SUNNY
SUNNY
CLOUDY
RAINY
FOG
FOG
CLOUDY
CLOUDY
RAINY
SNOW
FOG
CLOUDY
FOG
RAINY
SNOW
RAINY
FOG
FOG
SNOW
CLOUDY
SUNNY
SNOW
SNOW
SNOW
SNOW
CLOUDY
CLOUDY
SNOW
FOG
SNOW
CLOUDY
SUNNY
SNOW
SNOW
FOG
SUNNY
CLOUDY
FOG
RAINY
CLOUDY
FOG
SUNNY
CLOUDY
SUNNY
RAINY
FOG
SNOW
CLOUDY
RAINY
FOG
SNOW
RAINY
RAINY
SNOW
SNOW
FOG
CLOUDY
SNOW
SNOW
SUNNY
CLOUDY
CLOUDY
SNOW
FOG
SNOW
SNOW
SNOW
FOG
FOG
SNOW
SUNNY
FOG
CLOUDY
FOG
RAINY
FOG
SNOW
SNOW
FOG
FOG
RAINY
RAINY
SNOW
RAINY
FOG
FOG


# Cleanup